In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as style
import matplotlib.dates as mdates
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import seaborn as sns
sns.set_style("darkgrid")
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.trend import MACD
from ta.volatility import BollingerBands
from datetime import datetime, timedelta
SP_500_database = pd.read_csv("database2.csv")

Everytime you edit any cell click run on that cell

# user inputs: 

In [84]:
ticker = "AAPL" # Uppercase stock ticker in the S&P 500

start_date = '01-01-2017' # Enter start date in format 'MM-DD-YYYY'
end_date = '01-01-2020' # Enter end date in format 'MM-DD-YYYY'

enable_RSI = True # True to filter based on RSI condition, False to not filter
RSI_period = 14 # parameter period for RSI
RSI_conditional_value = 35  # To be considered a falling knife RSI has to be outside of this value (oversold value)
show_RSI_overbought_line = True # shows a straight line to visualize the overbought value (100 - RSI_conditional_value)
show_RSI_oversold_line = True # shows a straight line to visualize the oversold value (RSI_conditional_value)

enable_MACD = False # True to filter based on MACD condition, False to not filter
MACD_short_term_period = 12 # parameter for short term period for MACD
MACD_long_term_period = 26 # parameter for long term period for MACD
MACD_signal_period = 9 # parameter for signal period for MACD
MACD_conditional_value = -.01 # To be considered a falling knife MACD has to be outside of this value
show_MACD_signal_line = True # True to graph the MACD signal line

enable_RETURN = False # True to filter and graph based on return condition, False to not filter
RETURN_conditional_value = -.025 # To be considered a falling knife the percentage change of the stock has to be less than this value

enable_BOLLINGER_BANDS = False # True to filter based on condition, False to not filter
BOLLINGER_BANDS_period = 20 # number of days in smoothing period (usually 20)
BOLLINGER_BANDS_std = 2 # number of standard deviations (usually 2)
show_BOLLINGER_BANDS = True # Display Bollinger Bands on closing price g

enable_SMA = False # if SMA is less than price consider it a falling knife
SMA_period = 7
show_SMA = True

graph_closing_price = False # True to graph, False to not graph

In [96]:
company_name = SP_500_database[SP_500_database["Ticker"]==ticker]['Company']
company_name

KeyError: 0

# user should only click run on the cells below.  DO NOT EDIT CELLS BELOW UNLESS YOU KNOW WHAT YOU ARE EDITING

You only have to run this first cell once

Run this next cell every time you edit some thing in the user input cell

In [85]:
# cleans and reformats the database based on the ticker
ticker_database = SP_500_database[SP_500_database["Ticker"]==ticker]
# company_name = ticker_database['Company'][0]
ticker_database = ticker_database.round(2)
ticker_database['Date'] = pd.to_datetime(ticker_database['Date'])
ticker_database = ticker_database.set_index('Date')

#print(ticker_database.head(-5))
#filters database based on your choosen start and end date
#print(ticker_database.head(-5))



# creates indicator data
ticker_database['MACD'] = MACD(ticker_database["Adj Close"], n_slow=MACD_short_term_period, n_fast=MACD_long_term_period, n_sign=MACD_signal_period).macd()
ticker_database['MACD Signal'] = MACD(ticker_database["Adj Close"], n_slow=MACD_short_term_period, n_fast=MACD_long_term_period, n_sign=MACD_signal_period).macd_signal()

ticker_database['RSI'] = RSIIndicator(ticker_database["Adj Close"], RSI_period).rsi()

ticker_database['Return'] = ticker_database["Adj Close"].pct_change()

BB = BollingerBands(ticker_database['Adj Close'], BOLLINGER_BANDS_period, BOLLINGER_BANDS_std)
ticker_database['Bollinger Bands High'] = BB.bollinger_hband()
ticker_database['Bollinger Bands Low'] = BB.bollinger_lband()

ticker_database['SMA'] = ticker_database['Adj Close'].rolling(window=SMA_period).mean()

ticker_database = ticker_database.loc[start_date:end_date]

ticker_database_filtered = ticker_database

# these are the condition statements below: I couldnt figure out how to keep them in one line so..
print("\nfiltering data based on conditions...")
if enable_RSI:
    ticker_database_filtered = ticker_database_filtered[ticker_database_filtered['RSI'] < RSI_conditional_value]

if enable_MACD:
    ticker_database_filtered = ticker_database_filtered[ticker_database_filtered["MACD"] <= MACD_conditional_value]

if enable_RETURN:
    ticker_database_filtered = ticker_database_filtered[ticker_database_filtered["Return"] <= RETURN_conditional_value]

if enable_SMA:
    ticker_database_filtered = ticker_database_filtered[ticker_database_filtered["Adj Close"] <= ticker_database_filtered["SMA"]]


# fallingKniveDf = df[df['Adj Close'] < df['SMA']]

print("There are " + str(len(ticker_database_filtered)) + " falling knives")
print(ticker_database_filtered)
'''
# takes the filtered df from above and graphs a specific interval around that date
print("\nCreating new dataframes...")
aroundIndex = []
for date in ticker_database_filtered.index:
    aroundIndex.append(pd.date_range(start=date - timedelta(weeks=window/2), end=date+ timedelta(weeks=window/2)))

FallingKnifeSubset = []
for dates in aroundIndex:
    FallingKnifeSubset.append(df[df.index.isin(dates)])
'''



filtering data based on conditions...
There are 45 falling knives
           Ticker     Company    Open   Close  Adj Close    Return    High  \
Date                                                                         
2017-06-12   AAPL  Apple Inc.  145.74  145.42     139.76 -0.023886  146.09   
2017-06-15   AAPL  Apple Inc.  143.32  144.29     138.67 -0.006021  144.48   
2017-06-16   AAPL  Apple Inc.  143.78  142.27     136.73 -0.013990  144.50   
2017-09-22   AAPL  Apple Inc.  151.54  151.89     146.55 -0.009797  152.27   
2017-09-25   AAPL  Apple Inc.  149.99  150.55     145.26 -0.008802  151.83   
2018-01-29   AAPL  Apple Inc.  170.16  167.96     162.64 -0.020713  170.16   
2018-01-30   AAPL  Apple Inc.  165.53  166.97     161.68 -0.005903  167.37   
2018-01-31   AAPL  Apple Inc.  166.87  167.43     162.13  0.002783  168.44   
2018-02-02   AAPL  Apple Inc.  166.00  160.50     155.42 -0.043393  166.80   
2018-02-05   AAPL  Apple Inc.  159.10  156.49     151.53 -0.025029  163.88 

'\n# takes the filtered df from above and graphs a specific interval around that date\nprint("\nCreating new dataframes...")\naroundIndex = []\nfor date in ticker_database_filtered.index:\n    aroundIndex.append(pd.date_range(start=date - timedelta(weeks=window/2), end=date+ timedelta(weeks=window/2)))\n\nFallingKnifeSubset = []\nfor dates in aroundIndex:\n    FallingKnifeSubset.append(df[df.index.isin(dates)])\n'

This next cell manages the graphing

In [88]:
# Create figure
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, row_heights=[.2, .2,0.6])

fig.add_trace(go.Scatter(x=ticker_database.index, y=ticker_database['RSI'],
                    name='RSI'),
                    row=1, col=1)

fig.add_trace(go.Scatter(x=ticker_database_filtered.index, y=ticker_database_filtered['RSI'],
                    mode='markers',
                    name='fn RSI'),
                    row=1, col=1)
if show_RSI_overbought_line:
    fig.add_trace(go.Scatter(x=[ticker_database.index.min(), ticker_database.index.max()], y=[100 - RSI_conditional_value, 100 - RSI_conditional_value],
                    name='RSI overbought'),
                    row=1, col=1)
if show_RSI_oversold_line:
    fig.add_trace(go.Scatter(x=[ticker_database.index.min(), ticker_database.index.max()], y=[RSI_conditional_value, RSI_conditional_value],
                    name='RSI oversold'),
                    row=1, col=1)


fig.add_trace(go.Scatter(x=ticker_database.index, y=ticker_database['MACD'], 
                    name='MACD'), 
                    row=2, col=1)
if show_MACD_signal_line:
    fig.add_trace(go.Scatter(x=ticker_database.index, y=ticker_database['MACD Signal'], 
                    name='MACD Signal'), 
                    row=2, col=1)


fig.add_trace(go.Scatter(x=ticker_database_filtered.index, y=ticker_database_filtered['MACD'],
                    mode='markers',
                    name='fn MACD'),
                    row=2, col=1)



fig.add_trace(go.Scatter(x=ticker_database.index, y=ticker_database['Adj Close'],
                    name='Closing Price'), 
                    row=3, col=1)

fig.add_trace(go.Scatter(x=ticker_database_filtered.index, y=ticker_database_filtered['Adj Close'],
                    mode='markers',
                    name='fn Closing Price'),
                    row=3, col=1)

if show_BOLLINGER_BANDS:
    fig.add_trace(go.Scatter(x=ticker_database.index, y=ticker_database['Bollinger Bands High'],
                    name='Bollinger Bands High'),
                    row=3, col=1)

    fig.add_trace(go.Scatter(x=ticker_database_filtered.index, y=ticker_database_filtered['Bollinger Bands Low'],
                    name='Bollinger Bands High'),
                    row=3, col=1)

if show_SMA:
    fig.add_trace(go.Scatter(x=ticker_database_filtered.index, y=ticker_database_filtered['SMA'],
                    name='SMA'),
                    row=3, col=1)




# Add range slider
fig.update_layout(
    showlegend=False,
    title_text=ticker + ": " + ,
    yaxis_title="RSI",
    yaxis2_title="MACD",
    yaxis3_title="Closing Price",
    xaxis3_title="Timeframe",
    xaxis3_rangeslider_visible=True,
    height=1000,
    width=1200,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=3,
                     label="3m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    )
)

fig.show()

MA
Bolling bands*
fibonici sequence better
elliot wave
price to volume ratio
oob
adi
*vwap: where the price starts to accumate, ex: does 1 person by 100,000 shares or does (P is above or below vwap)
put and call ratio

letting the user graph
lines, circles



9 21 45 90
normal distubition
3 version, extreme, moderate, light
